In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath("/home/michaelshewarega/Desktop/test/arkouda"))
import arkouda as ak
import arachne as ar
import pandas as pd
import numpy as np
import time
import json

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.06.21


In [2]:
ak.connect()

connected to arkouda server tcp://*:5555


In [3]:
# These are just wedges, we do not need subgraph isomorphism for these.
src0 = [    1,     1]
dst0 = [10002, 10003]
connection_type0 = ['n', 'n']
color0 = ['b', 'b']

src1 = [    1,     1, 10003, 10003]
dst1 = [10002, 10003, 30004, 30005]
connection_type1 = ['n', 'n', 'n', 'n']
color1 = ['b', 'b', 'b', 'b']

src2 = [    1,     1, 40005]
dst2 = [10002, 10003,     1]
connection_type2 = ['n', 'n', 's']
color2 = ['b', 'b', 'b_r']

src3 = [    1,     1, 40005, 40005, 50008]
dst3 = [10002, 10003, 50008,     1, 10003]
connection_type3 = ['n', 'n', 'n', 's', 's']
color3 = ['b', 'b', 'r', 'b_r', 'b_r']

src4 = [1, 1, 10003, 10003, 60007]
dst4 = [10002, 10003, 30004, 30005, 1]
connection_type4 = ['n', 'n', 'n', 'n', 's']
color4 = ['b', 'b', 'b', 'b', 'b_r']

src5 = [1, 1, 10003, 10003, 60007, 60007, 70010]
dst5 = [10002, 10003, 30004, 30005, 70010, 1, 30005]
connection_type5 = ['n', 'n', 'n', 'n', 'n', 's', 's']
color5 = ['b', 'b', 'b', 'b', 'r', 'b_r',  'b_r']

src6 = [1, 1, 40005, 40005, 80009, 80009, 10003, 90010]
dst6 = [10002, 10003, 50006, 50007, 90010, 90011, 50006, 50007]
connection_type6 = ['n', 'n', 'n', 'n', 'n', 'n', 's', 's']
color6 = ['b', 'b', 'r', 'r', 'y', 'y', 'b_r', 'b_y']

src7 = [1, 10002, 40005, 60007, 80009]
dst7 = [10002, 20003, 1, 10002, 20003]
connection_type7 = ['n', 'n', 's', 's', 's']
color7 = ['b', 'b', 'b_r', 'b_y', 'r_y']

subgraphs_as_list_of_tuples = [ (src0, dst0, connection_type0, color0, "0"),
                                (src1, dst1, connection_type1, color1, "1"),
                                (src2, dst2, connection_type2, color2, "2"),
                                (src3, dst3, connection_type3, color3, "3"),
                                (src4, dst4, connection_type4, color4, "4"),
                                (src5, dst5, connection_type5, color5, "5"),
                                (src6, dst6, connection_type6, color6, "6"),
                                (src7, dst7, connection_type7, color7, "7")
                            ]

In [4]:
df = pd.read_csv("/home/michaelshewarega/Desktop/test/OL_random_2.csv")
ak.connect()
transformed_dataset = ak.DataFrame(df.to_dict(orient='list'))

/tmp/ipykernel_385612/3300084937.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/home/michaelshewarega/Desktop/test/OL_random_2.csv")
connected to arkouda server tcp://*:5555


In [5]:
g = ar.PropGraph()
g.load_edge_attributes(transformed_dataset, source_column="src", destination_column="dst", 
                            relationship_columns=["s_bef", "s_bef_x", "s_bef_y", "s_bef_z", "s_af", "s_af_x", 
                                                    "s_af_y", "s_af_z", 's_x', "s_y", "s_z", "s_distance", "d_bef",
                                                    "d_bef_x", "d_bef_y", "d_bef_z", "d_af", "d_af_x", "d_af_y", "d_af_z",
                                                    "d_x", "d_y", "d_z", "d_distance", "n_id", "connection_type"])

print(f"Built graph with {len(g)} vertices and {g.size()} edges")

Built graph with 667711 vertices and 962796 edges


In [13]:
def get_mapping(g, subgraph, iso_cap):
    # Process subgraph information for mapping after subgraph isomorphism is invoked.
    src_sub, dst_sub = subgraph.edges()
    src_sub = src_sub.to_ndarray()
    dst_sub = dst_sub.to_ndarray()
    subgraph_nodes = sorted(list(np.unique(np.concatenate((src_sub, dst_sub)))))

    start = time.time()
    if iso_cap > 0 :
        isos = ar.subgraph_isomorphism(g, subgraph, algorithm_type="si", return_isos_as="complete", semantic_check="or", size_limit=iso_cap)
    else:
        isos = ar.subgraph_isomorphism(g, subgraph, algorithm_type="si", return_isos_as="complete", semantic_check="or")
    end = time.time()
    
    # Extract the returned array information from subgraph_isomorphism.
    isos_by_vertices = isos[0]
    isos_by_vertices_map = isos[1]
    isos_by_edges_src = isos[2]
    isos_by_edges_dst = isos[3]

    if len(isos_by_vertices) % len(subgraph) != 0:
        raise ValueError("The length of isomorphisms is not a multiple of the number of subgraph nodes.")

    # Get the number of motifs found.
    number_isos_found = len(isos_by_vertices) // len(subgraph_nodes)
    print(f"Finding {number_isos_found:_} motifs took: {end-start} seconds.")

    # Prepare the returned isomorphisms as a 2D array.
    start = time.time()
    isos_ndarray = isos_by_vertices.to_ndarray()
    hostgraph_nodes = isos_ndarray.reshape(-1, len(subgraph_nodes))
    end = time.time()
    print(f"Reshaping isomorphisms took: {end-start} seconds.")

    # Create all mappings at once using a list comprehension.
    start = time.time()
    all_mappings = [
        {int(k): int(v) for k, v in zip(subgraph_nodes, hostgraph_nodes[i])}
        for i in range(number_isos_found)
    ]
    end = time.time()
    print(f"Generating mappings took: {end-start} seconds.")

    return (all_mappings,(isos_by_edges_src,isos_by_edges_dst))

In [7]:
def motif_to_viss(src, dst, connection_type, iso_cap):    
    subgraph_dict = {
        "src": src,
        "dst": dst,
        "connection_type": connection_type
        }
    
    subgraph = ar.PropGraph()
    df = ak.DataFrame(subgraph_dict)
    subgraph.load_edge_attributes(df, source_column="src", destination_column="dst", 
                                relationship_columns=["connection_type"])
    
    node_mapping = get_mapping(g, subgraph, iso_cap)
    return node_mapping

In [8]:
send_to_file = False
all_mappings = []
for tup in subgraphs_as_list_of_tuples:
    subgraph_name = f"subgraph{tup[4]}"
    cap = 5
    print(f"Querying for {subgraph_name}...")
    curr_mapping = motif_to_viss(tup[0], tup[1], tup[2], cap)
    all_mappings.append(curr_mapping)
    print()

    if send_to_file:
        filename = subgraph_name + f"_motifs_capped_at_{cap}.json"
        with open(filename, "w") as f:
            json.dump(curr_mapping[0], f)

Querying for subgraph0...
Finding 7 motifs took: 4.513763904571533 seconds.
Reshaping isomorphisms took: 0.0005924701690673828 seconds.
Generating mappings took: 2.1457672119140625e-05 seconds.

Querying for subgraph1...
Finding 6 motifs took: 5.051302671432495 seconds.
Reshaping isomorphisms took: 0.0007410049438476562 seconds.
Generating mappings took: 3.457069396972656e-05 seconds.

Querying for subgraph2...
Finding 6 motifs took: 4.440776824951172 seconds.
Reshaping isomorphisms took: 0.0005822181701660156 seconds.
Generating mappings took: 2.0265579223632812e-05 seconds.

Querying for subgraph3...
Finding 5 motifs took: 4.401183605194092 seconds.
Reshaping isomorphisms took: 0.0006394386291503906 seconds.
Generating mappings took: 1.811981201171875e-05 seconds.

Querying for subgraph4...
Finding 6 motifs took: 4.846199989318848 seconds.
Reshaping isomorphisms took: 0.0007688999176025391 seconds.
Generating mappings took: 3.8623809814453125e-05 seconds.

Querying for subgraph5...
F

In [12]:
subgraphs_as_list_of_tuples

[([1, 1], [10002, 10003], ['n', 'n'], ['b', 'b'], '0'),
 ([1, 1, 10003, 10003],
  [10002, 10003, 30004, 30005],
  ['n', 'n', 'n', 'n'],
  ['b', 'b', 'b', 'b'],
  '1'),
 ([1, 1, 40005], [10002, 10003, 1], ['n', 'n', 's'], ['b', 'b', 'b_r'], '2'),
 ([1, 1, 40005, 40005, 50008],
  [10002, 10003, 50008, 1, 10003],
  ['n', 'n', 'n', 's', 's'],
  ['b', 'b', 'r', 'b_r', 'b_r'],
  '3'),
 ([1, 1, 10003, 10003, 60007],
  [10002, 10003, 30004, 30005, 1],
  ['n', 'n', 'n', 'n', 's'],
  ['b', 'b', 'b', 'b', 'b_r'],
  '4'),
 ([1, 1, 10003, 10003, 60007, 60007, 70010],
  [10002, 10003, 30004, 30005, 70010, 1, 30005],
  ['n', 'n', 'n', 'n', 'n', 's', 's'],
  ['b', 'b', 'b', 'b', 'r', 'b_r', 'b_r'],
  '5'),
 ([1, 1, 40005, 40005, 80009, 80009, 10003, 90010],
  [10002, 10003, 50006, 50007, 90010, 90011, 50006, 50007],
  ['n', 'n', 'n', 'n', 'n', 'n', 's', 's'],
  ['b', 'b', 'r', 'r', 'y', 'y', 'b_r', 'b_y'],
  '6'),
 ([1, 10002, 40005, 60007, 80009],
  [10002, 20003, 1, 10002, 20003],
  ['n', 'n', 's', 

In [9]:
for tup in subgraphs_as_list_of_tuples:
    subgraph_id = int(tup[4])
    subgraph_name = f"subgraph{subgraph_id}"
    subgraph_src = tup[0]
    subgraph_dst = tup[1]
    color = tup[3]

    dict_to_check_against = {}
    for c in color:
        if c not in dict_to_check_against:
            dict_to_check_against[c] = 1
        else:
            dict_to_check_against[c] += 1

    print(f"Post processing {subgraph_name}...")

    isos_by_vertices = all_mappings[subgraph_id][0]
    isos_by_edges_src = all_mappings[subgraph_id][1][0]
    isos_by_edges_dst = all_mappings[subgraph_id][1][1]
    
    num_edges_subgraph = len(subgraph_src)
    number_isos_found = len(isos_by_edges_src) // len(subgraph_src)

    indices = ak.find([isos_by_edges_src,isos_by_edges_dst],[g.edge_attributes["src"], g.edge_attributes["dst"]])
    vals = g.edge_attributes["n_id"][indices]

    # TODO: Can the below be done with Arkouda?
    isos_by_edges_src_ndarray = np.split(isos_by_edges_src.to_ndarray(), number_isos_found)
    isos_by_edges_dst_ndarray = np.split(isos_by_edges_dst.to_ndarray(), number_isos_found)
    vals_ndarray = np.split(vals.to_ndarray(), number_isos_found)

    matches = 0
    curr_mapping_id = 0
    for src,dst,vals in zip(isos_by_edges_src_ndarray,isos_by_edges_dst_ndarray,vals_ndarray):
        if subgraph_id == 0 or subgraph_id == 1:
            if len(np.unique(vals)) == 1:
                matches += 1
                continue
        else:
            inner_matches = {}
            for val in vals:
                if val not in inner_matches:
                    inner_matches[val] = 1
                else:
                    inner_matches[val] += 1
            if sorted(inner_matches.values()) == sorted(dict_to_check_against.values()):
                matches += 1
            # else:
            #     print(isos_by_vertices[curr_mapping_id])
            #     print(src)
            #     print(dst)
            curr_mapping_id += 1

    print(f"Before post processing number of motifs found was {number_isos_found} and after was {matches}")
    print()

Post processing subgraph0...
Before post processing number of motifs found was 7 and after was 7

Post processing subgraph1...
Before post processing number of motifs found was 6 and after was 6

Post processing subgraph2...
Before post processing number of motifs found was 6 and after was 6

Post processing subgraph3...
Before post processing number of motifs found was 5 and after was 5

Post processing subgraph4...
Before post processing number of motifs found was 6 and after was 6

Post processing subgraph5...
Before post processing number of motifs found was 5 and after was 5

Post processing subgraph6...
Before post processing number of motifs found was 5 and after was 5

Post processing subgraph7...
Before post processing number of motifs found was 6 and after was 2



In [13]:
all_mappings[7]

([{1: 1168310597770406,
   10002: 639052900814641,
   20003: 486235902793191,
   40005: 171355985676791,
   60007: 314633194424895,
   80009: 1217578650836844},
  {1: 1168310597770406,
   10002: 639052900814641,
   20003: 486235902793191,
   40005: 30275063655515,
   60007: 314633194424895,
   80009: 1217578650836844},
  {1: 1168310597770406,
   10002: 639052900814641,
   20003: 486235902793191,
   40005: 489156294457428,
   60007: 314633194424895,
   80009: 1217578650836844},
  {1: 1168310597770406,
   10002: 639052900814641,
   20003: 486235902793191,
   40005: 171355985676791,
   60007: 1210670403173151,
   80009: 1217578650836844},
  {1: 1168310597770406,
   10002: 639052900814641,
   20003: 486235902793191,
   40005: 30275063655515,
   60007: 1210670403173151,
   80009: 1217578650836844},
  {1: 1168310597770406,
   10002: 639052900814641,
   20003: 486235902793191,
   40005: 489156294457428,
   60007: 1210670403173151,
   80009: 1217578650836844}],
 (array([1168310597770406 639052